In [2]:
import pandas as pd
from scipy.signal import savgol_filter
from sklearn.preprocessing import MinMaxScaler


In [8]:
#read files
train_file=r'C:\Users\65962\Desktop\JUPYTER\CMAPSSData\train_FD001.txt'
test_file=r"C:\Users\65962\Desktop\JUPYTER\CMAPSSData\test_FD001.txt"
RUL_file=r"C:\Users\65962\Desktop\JUPYTER\CMAPSSData\RUL_FD001.txt"

column_names = ['unit_number', 'time_in_cycles', 'operational_setting_1', 
                'operational_setting_2', 'operational_setting_3'] + \
                [f'sensor_{i}' for i in range(1, 22)]
train_df = pd.read_csv(train_file, sep=' ', header=None, names=column_names)
test_df = pd.read_csv(test_file, sep=' ', header=None, names=column_names)
RUL_df = pd.read_csv(RUL_file, header=None, names=['RUL'])

#remove sensors?
irrelevant_sensors = ['sensor_3']
train_df = train_df.drop(columns=irrelevant_sensors)
test_df = test_df.drop(columns=irrelevant_sensors)

In [9]:
sensor_columns = [col for col in train_df.columns if 'sensor' in col]
for sensor in sensor_columns:
    train_df[sensor] = savgol_filter(train_df[sensor], window_length=11, polyorder=2)
    test_df[sensor] = savgol_filter(test_df[sensor], window_length=11, polyorder=2)



In [11]:
scaler = MinMaxScaler()
train_df[sensor_columns] = scaler.fit_transform(train_df[sensor_columns])
test_df[sensor_columns] = scaler.transform(test_df[sensor_columns])  # Use the same scaler
test_rul_df = test_df.groupby('unit_number').last().reset_index()
test_rul_df['RUL'] = RUL_df['RUL'] 
print(train_df.head())
print(test_df.head())


     unit_number  time_in_cycles  operational_setting_1  \
1 1      -0.0007         -0.0004                  100.0   
  2       0.0019         -0.0003                  100.0   
  3      -0.0043          0.0003                  100.0   
  4       0.0007          0.0000                  100.0   
  5      -0.0019         -0.0002                  100.0   

     operational_setting_2  operational_setting_3  sensor_1  sensor_2  \
1 1                 518.67                 641.82  0.393393  0.288209   
  2                 518.67                 642.15  0.362438  0.277299   
  3                 518.67                 642.35  0.336909  0.266624   
  4                 518.67                 642.35  0.316806  0.256183   
  5                 518.67                 642.37  0.302131  0.245977   

     sensor_4  sensor_5  sensor_6  ...  sensor_12  sensor_13     sensor_14  \
1 1  0.806971  0.757127  0.311686  ...   0.192367   0.341494  3.122502e-17   
  2  0.806971  0.757059  0.289994  ...   0.180739 

C:\Users\65962\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
C:\Users\65962\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
